# FTP Transfer of NYSE, NASDAQ and Other Market Tickers

In [1]:
from ftplib import FTP
import pandas_datareader.data as web
from datetime import datetime, timedelta
import pandas as pd
import pickle
import yfinance as yf
import sqlite3 as sq3

In [2]:
directory = 'symboldirectory'
filenames = ('otherlisted.txt', 'nasdaqlisted.txt')

ftp = FTP('ftp.nasdaqtrader.com')
ftp.login()
ftp.cwd(directory)

for item in filenames:
    ftp.retrbinary('RETR {0}'.format(item), open(item, 'wb').write)

ftp.quit()

# Create pandas dataframes from the nasdaqlisted and otherlisted files.
nasdaq_exchange_info = pd.read_csv('nasdaqlisted.txt', '|')
other_exchange_info = pd.read_csv('otherlisted.txt', '|')

C:\Users\Eddie\anaconda3\envs\securities-analysis\lib\site-packages\IPython\core\interactiveshell.py:3441: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
nasdaq_exchange_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4990 entries, 0 to 4989
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Symbol            4990 non-null   object 
 1   Security Name     4989 non-null   object 
 2   Market Category   4989 non-null   object 
 3   Test Issue        4989 non-null   object 
 4   Financial Status  4989 non-null   object 
 5   Round Lot Size    4989 non-null   float64
 6   ETF               4980 non-null   object 
 7   NextShares        4989 non-null   object 
dtypes: float64(1), object(7)
memory usage: 312.0+ KB


In [4]:
nasdaq_exchange_info

,Symbol,Security Name,Market Category,Test Issue,Financial Status,Round Lot Size,ETF,NextShares
0,AACG,ATA Creativity Global - American Depositary Sh...,G,N,N,100.0,N,N
1,AACIU,Armada Acquisition Corp. I - Unit,G,N,N,100.0,N,N
2,AADI,"Aadi Bioscience, Inc. - Common Stock",S,N,N,100.0,N,N
3,AADR,AdvisorShares Dorsey Wright ADR ETF,G,N,N,100.0,Y,N
4,AAL,"American Airlines Group, Inc. - Common Stock",Q,N,N,100.0,N,N
...,...,...,...,...,...,...,...,...
4985,ZXZZT,NASDAQ TEST STOCK,G,Y,N,100.0,NaN,N
4986,ZY,Zymergen Inc. - Common Stock,Q,N,N,100.0,N,N
4987,ZYNE,"Zynerba Pharmaceuticals, Inc. - Common Stock",G,N,N,100.0,N,N
4988,ZYXI,"Zynex, Inc. - Common Stock",Q,N,N,100.0,N,N


In [5]:
# contains the tickers from NYSE MKT (Exchange = A),
# New York Stock Exchange (NYSE) (Exchange = N),
# NYSE ARCA (Exchange = P),
# BATS Global Markets (Exchange = BATS),
# Investors' Exchange (Exchange = IEXG)
other_exchange_info

,ACT Symbol,Security Name,Exchange,CQS Symbol,ETF,Round Lot Size,Test Issue,NASDAQ Symbol
0,A,"Agilent Technologies, Inc. Common Stock",N,A,N,100.0,N,A
1,AA,Alcoa Corporation Common Stock,N,AA,N,100.0,N,AA
2,AAA,Listed Funds Trust AAF First Priority CLO Bond...,P,AAA,Y,100.0,N,AAA
3,AAAU,Goldman Sachs Physical Gold ETF Shares,P,AAAU,Y,100.0,N,AAAU
4,AAC,Ares Acquisition Corporation Class A Ordinary ...,N,AAC,N,100.0,N,AAC
...,...,...,...,...,...,...,...,...
6130,ZVIA,Zevia PBC Class A Common Stock,N,ZVIA,N,100.0,N,ZVIA
6131,ZVV,NYSE ARCA test stock,P,ZVV,N,100.0,Y,ZVV
6132,ZXIET,IEX Test Company Test Symbol Three for IEX,V,ZXIET,N,100.0,Y,ZXIET
6133,ZYME,Zymeworks Inc. Common Shares,N,ZYME,N,100.0,N,ZYME


### Find all stock symbols that are beating the S&P 500 by more 
### than 10% over the last 365 days and store the last 365 days worth
### of data for those stock tickers into a python pickle.
### Use SPY for the ETF which tracks the S&P 500

In [6]:
end = datetime.now()
start = end - timedelta(days=365)

valid_sources = ('morningstar', 'iex', 'yahoo')
close_string = 'Close'
source = valid_sources[2]

spy = web.DataReader('SPY', source, start, end)
spy_percent_change = (spy.iloc[-1][close_string] - spy.iloc[0][close_string]) / \
    spy.iloc[0][close_string] * 100

In [7]:
print(f"Esseentially, S&P 500 has changed by {round(spy_percent_change,2)}% over the last year")

Esseentially, S&P 500 has changed by 30.57% over the last year


In [16]:
# list of all the tickers in the nasdaq

nasdaq_tickers = list(nasdaq_exchange_info['Symbol'])

In [18]:
# Loop through NASDAQ tickers and store stocks which have performed greater than the S&P 500 by at least 10%
# in the last year

# test
#nasdaq_tickers = ['TSLA','AAPL']
end = datetime.now()
start = end - timedelta(days=365)

growth_stocks = dict()
for ticker in nasdaq_tickers:
    try:
        #print(f'Checking stock: {ticker}')
        data = yf.download(ticker, start, end, progress=False)
        last_close = data['Close'][-1]
        if last_close > 10.0: # stocks cheaper than this are sketch
            start_close = data['Close'][0]
            percent_change = (last_close - start_close) / start_close * 100
            #print(f"Percent change of {ticker} in the last year is: {round(percent_change,2)}%")
            #print(f"Difference between this annual total return and that of S&P 500 is: \
            #{round(percent_change - spy_percent_change, 2)}%")
            if percent_change - spy_percent_change > 10:
                growth_stocks[ticker] = data
                #print(growth_stocks.keys())
    except Exception as e:
        print(e)

pickle.dump(growth_stocks, open('../../../data/nasdaq_growth_2021.p', 'wb'))

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

KeyboardInterrupt: 

In [10]:
growth_stocks

{}

In [29]:
# download the whole daily history of the nasdaq
# SLOWS DOWN COMPUTER
#historical_data = yf.download(nasdaq_tickers_str)

historical_data = yf.download(nasdaq_tickers)

[***                    7%                       ]  373 of 4990 completed

KeyboardInterrupt: 

[****                   8%                       ]  389 of 4990 completed

In [15]:
historical_data.head()

Adj Close          Close           High            Low       \
                AAPL TSLA      AAPL TSLA      AAPL TSLA      AAPL TSLA   
Date                                                                     
1980-12-12  0.100600  NaN  0.128348  NaN  0.128906  NaN  0.128348  NaN   
1980-12-15  0.095352  NaN  0.121652  NaN  0.122210  NaN  0.121652  NaN   
1980-12-16  0.088353  NaN  0.112723  NaN  0.113281  NaN  0.112723  NaN   
1980-12-17  0.090540  NaN  0.115513  NaN  0.116071  NaN  0.115513  NaN   
1980-12-18  0.093165  NaN  0.118862  NaN  0.119420  NaN  0.118862  NaN   

                Open          Volume       
                AAPL TSLA       AAPL TSLA  
Date                                       
1980-12-12  0.128348  NaN  469033600  NaN  
1980-12-15  0.122210  NaN  175884800  NaN  
1980-12-16  0.113281  NaN  105728000  NaN  
1980-12-17  0.115513  NaN   86441600  NaN  
1980-12-18  0.118862  NaN   73449600  NaN

In [20]:
historical_data = historical_data.stack(1).reset_index().rename(columns = {'level_1':'Ticker'}).set_index('Date')
historical_data.head()

In [18]:
con = sq3.connect('../../../data/nasdaq_history.sql')
historical_data.to_sql('nasdaq_hist', con)

C:\Users\Eddie\anaconda3\envs\securities-analysis\lib\site-packages\pandas\core\generic.py:2872: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


OperationalError: too many columns on nasdaq_hist